# 1. Trabalho Prático 2 - Análise de dados de Obras não publicitárias registradas pela Agência Nacional de Cinema

# 2. Membros

Gabriel Camatta Zanotelli - 2018020140 

Yaggo Victor Cardoso de Souza Teixeira - 2021086474

Cayo Victor Rocha Guzansky - 2021070802

# 3. Descrição dos dados

Os dados utilizados foram retirados do [portal de dados](https://dados.gov.br/dados/conjuntos-dados/crt-obras-nao-publicitarias-registradas) do governo federal. O conjunto de dados selecionado contém informações como nome, tipo, país de origem e requerente de obras não publicitárias registradas pela Ancine entre 2014 e novembro de 2025.

## Processamento dos dados

## Processamento inicial

### Bibliotecas e funções auxiliares

In [ ]:
import csv
import pandas as pd
import sqlite3

import requests
from PIL import Image
from io import BytesIO
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
def fetch(query, conn, formatted=True):
    # execute the query and fetch all rows
    cur = conn.cursor()
    cur.execute(query)
    rs = cur.fetchall()
    
    # extract column names from the cursor description
    columns = [desc[0] for desc in cur.description]
    
    # return a dataframe with column names
    return pd.DataFrame(rs, columns=columns) if formatted else rs

def show_tables(conn):
    return [x[0] for x in fetch('SELECT tbl_name FROM sqlite_master WHERE type="table"', conn, False)]

def shape(table, conn):
    nrows = fetch(f'SELECT COUNT(*) FROM {table}', conn, False)[0][0]
    ncols = fetch(f'SELECT COUNT(*) FROM pragma_table_info("{table}")', conn, False)[0][0]

    return (nrows, ncols)

def desc(table, conn):
    cur = conn.cursor()
    cur.execute(f'PRAGMA table_info("{table}")')
    columns = [row[1] for row in cur.fetchall()]
    
    return columns

def info(table, conn):
    # table constraints (domain, null, default, pk)
    df1 = fetch(f'PRAGMA table_info("{table}")', conn)
    columns = desc(table, conn)
    
    # entries per column
    counts = ', '.join([f'COUNT(*) AS "{column}"' for column in columns])
    df2 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df2.columns = ['count']
    
    # non-null entries per column
    counts = ', '.join([f'COUNT("{column}") AS "{column}"' for column in columns])
    df3 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df3.columns = ['notnull count']

    # unique non-null entries per column
    counts = ', '.join([f'COUNT(DISTINCT "{column}") AS "{column}"' for column in columns])
    df4 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df4.columns = ['unique count']
    
    return df1.merge(df2, left_on='name', right_index=True) \
            .merge(df3, left_on='name', right_index=True) \
            .merge(df4, left_on='name', right_index=True)

def display(urls, cols=5):
    # fetch images
    images = []
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            images.append(Image.open(BytesIO(response.content)))

    # calculate the number of rows
    rows = (len(images) + cols - 1) // cols  
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows))

    for i, ax in enumerate(axs.flat):
        if i < len(images):
            ax.imshow(images[i])
            ax.axis("off")  # Hide axes
        else:
            ax.axis("off")  # Hide unused subplots
    plt.tight_layout()
    plt.show()

def hist(data, xlabel='', ylabel='', bins='auto'):
    # convert list of tuples into dataframe
    df = pd.DataFrame(data, columns=['key', 'frequency'])
    # expand the data based on frequencies
    expanded = df.loc[df.index.repeat(df['frequency'])].reset_index(drop=True)

    # plot histogram
    sns.histplot(expanded['key'], bins=bins, kde=False)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=90)
    plt.show()

### Extração de dados

Os dados foram extraídos de um aquivo *ancime_dataset.csv*, que é uma junção dos outro 11 arquivos separados pro ano de requisição disponíveis na plataforma.

In [ ]:
# source dataset: https://dados.gov.br/dados/conjuntos-dados/crt-obras-nao-publicitarias-registradas
source = './ancine_dataset.csv'

# connect to SQLite database
conn = sqlite3.connect('ancine.db')
cur = conn.cursor()

# set PRAGMA options for faster ingestion
cur.execute('PRAGMA synchronous = OFF')
cur.execute('PRAGMA journal_mode = MEMORY')

# batch size for processing large datasets
BATCH_SIZE = 1000

# begin transaction for bulk ingestion
cur.execute('BEGIN TRANSACTION')

# read the CSV file and ingest in batches
with open(source, 'r', encoding="utf8") as f:
    reader = csv.reader(f, delimiter=';')
    header = next(reader)  # Read the header row

    # create table with appropriate column names
    columns = [f'"{column}"' for column in header]
    create = f'CREATE TABLE IF NOT EXISTS Source ({", ".join(columns)})'
    cur.execute(create)

    # prepare the INSERT query
    placeholders = ', '.join(['?'] * len(header))
    insert = f'INSERT INTO Source VALUES ({placeholders})'

    # process rows in batches
    batch = []
    for row in reader:
        batch.append(row)
        if len(batch) == BATCH_SIZE:
            # Execute batch insert
            cur.executemany(insert, batch)
            # Reset batch
            batch = []

    # insert any remaining rows
    if batch:
        cur.executemany(insert, batch)

# nullify empty values, so they are recognized as proper nulls
update = f'UPDATE Source SET ' + ','.join([f'{column} = NULLIF({column}, "")' for column in columns])
cur.execute(update)

# commit changes
conn.commit()

# reset PRAGMA options (optional but recommended for safety)
cur.execute('PRAGMA synchronous = FULL')
cur.execute('PRAGMA journal_mode = DELETE')

Com o código abaixo podemos verificar que temos 169773 linhas de dados e 22 colunas.

In [ ]:
shape('Source', conn)

In [ ]:
info('Source', conn)

### Limpeza de dados

Para nossa análise, foram excluídas todas as requisições feitas por pessoas físicas.

In [ ]:
# Remove todas as Obras requisitadas por pessoa física, já que vamos usar o CNPJ como a chave do Requerente
cur.execute(f'DELETE FROM Source WHERE CNPJ_REQUERENTE  = "PESSOA FÍSICA"')
conn.commit()

## Migração dos dados

Os dados foram agrupados em 4 tabelas: Requisição, Classificação, Obras e Requerentes.

In [ ]:
# Deleta todas as tabelas e seus dados, caso elas já existam
ddl = [
    'DROP TABLE IF EXISTS Requisicao;',
    'DROP TABLE IF EXISTS Classificacao;',
    'DROP TABLE IF EXISTS Obras;',
    'DROP TABLE IF EXISTS Requerentes;'
]

for q in ddl:
    cur.execute(q)

conn.commit()

In [ ]:
# Prepara os shcemas

ddl = [
    '''
    CREATE TABLE IF NOT EXISTS Requerentes (
        CNPJ_REQUERENTE TEXT PRIMARY KEY,
        REQUERENTE TEXT NOT NULL,
        REGISTRO_ANCINE_REQUERENTE TEXT,
        UF_REQUERENTE TEXT,
        MUNICIPIO_REQUERENTE TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Obras (
        CRT TEXT PRIMARY KEY,
        TITULO_ORIGINAL TEXT NOT NULL,
        TITULO_BRASIL TEXT,
        PAIS TEXT,
        ORGANIZACAO_TEMPORAL TEXT,
        DURACAO_TOTAL_MINUTOS INTEGER,
        ANO_PRODUCAO_INICIAL INTEGER,
        ANO_PRODUCAO_FINAL INTEGER,
        SEGMENTO TEXT,
        REDUCAO_ISENCAO TEXT,
        CPB_ROE TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Classificacao (
        CRT TEXT PRIMARY KEY,
        CLASSIFICACAO TEXT NOT NULL,
        TIPO_OBRA TEXT,
        SUBTIPO_OBRA TEXT,
        FOREIGN KEY (CRT) REFERENCES Obras(CRT)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Requisicao (
        CRT TEXT,
        CNPJ_REQUERENTE TEXT,
        DATA_REQUERIMENTO_CRT DATE,
        DATA_EMISSAO_CRT DATE,
        SITUACAO_CRT TEXT,
        PRIMARY KEY (CRT, CNPJ_REQUERENTE),
        FOREIGN KEY (CRT) REFERENCES Obras(CRT),
        FOREIGN KEY (CNPJ_REQUERENTE) REFERENCES Requerentes(CNPJ_REQUERENTE)
    );
    '''
]

for q in ddl:
    cur.execute(q)

conn.commit()

#### Migra Requerentes

In [ ]:
# migrate Requerentes
cur.execute('DELETE FROM Requerentes')
cur.execute('''
INSERT OR IGNORE INTO Requerentes (
    CNPJ_REQUERENTE, REQUERENTE, REGISTRO_ANCINE_REQUERENTE, UF_REQUERENTE, MUNICIPIO_REQUERENTE
)
SELECT DISTINCT
    CNPJ_REQUERENTE, REQUERENTE, REGISTRO_ANCINE_REQUERENTE, UF_REQUERENTE, MUNICIPIO_REQUERENTE
FROM Source
WHERE CNPJ_REQUERENTE IS NOT NULL AND CNPJ_REQUERENTE != '';''')

conn.commit()
info('Requerentes', conn)

#### Migra Obras

In [ ]:
# migrate Obras
cur.execute('DELETE FROM Obras')
cur.execute('''
INSERT INTO Obras (
    CRT, TITULO_ORIGINAL, TITULO_BRASIL, PAIS, 
    ORGANIZACAO_TEMPORAL, DURACAO_TOTAL_MINUTOS, ANO_PRODUCAO_INICIAL, 
    ANO_PRODUCAO_FINAL, SEGMENTO, REDUCAO_ISENCAO, CPB_ROE
)
SELECT DISTINCT
    CRT, TITULO_ORIGINAL, TITULO_BRASIL, PAIS,
    ORGANIZACAO_TEMPORAL, DURACAO_TOTAL_MINUTOS, ANO_PRODUCAO_INICIAL,
    ANO_PRODUCAO_FINAL, SEGMENTO, REDUCAO_ISENCAO, CPB_ROE
FROM Source
WHERE CRT IS NOT NULL AND CRT != '';''')

conn.commit()
info('Obras', conn)

### Migra Classificacao

In [ ]:
# migrate Classificacao
cur.execute('DELETE FROM Classificacao')
cur.execute('''
INSERT INTO Classificacao (
    CRT, CLASSIFICACAO, TIPO_OBRA, SUBTIPO_OBRA
)
SELECT DISTINCT
    CRT, CLASSIFICACAO, TIPO_OBRA, SUBTIPO_OBRA
FROM Source
WHERE CRT IS NOT NULL AND CRT != '' AND CLASSIFICACAO IS NOT NULL AND CLASSIFICACAO != '';''')

conn.commit()
info('Classificacao', conn)

### Migra Requisicao

In [ ]:
# migrate Requisicao
cur.execute('DELETE FROM Requisicao')
cur.execute('''
INSERT INTO Requisicao (
    CRT, CNPJ_REQUERENTE, DATA_REQUERIMENTO_CRT, DATA_EMISSAO_CRT, SITUACAO_CRT
)
SELECT DISTINCT
    s.CRT, s.CNPJ_REQUERENTE, s.DATA_REQUERIMENTO_CRT, s.DATA_EMISSAO_CRT, s.SITUACAO_CRT
FROM Source s
WHERE s.CRT IS NOT NULL AND s.CRT != '' 
  AND s.CNPJ_REQUERENTE IS NOT NULL AND s.CNPJ_REQUERENTE != '';''')

conn.commit()
info('Requisicao', conn)

### Deleta tabela de origem

In [ ]:
# drop source table
cur.execute('DROP TABLE Source;')
conn.commit()

show_tables(conn)

# 4. Diagrama ER

![Diagrama ER](./diagrama_er.png)

# 5. Diagrama relacional

# 6. Consultas

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1

### 6.1.2 Consulta 2

## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3

### 6.2.2 Consulta 4

### 6.2.3 Consulta 5

## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Consulta 6

### 6.3.2 Consulta 7

### 6.3.3 Consulta 8

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9

### 6.4.2 Consulta 10

# 7. Autoavaliação dos membros

### Finalização do ambiente

In [ ]:
cur.close()
conn.close()